In [1]:
import os
import numpy as np
import cv2
import pandas as pd
from glob import glob
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import shutil
import json


# Get list of image path and coresponding groundtruth

In [2]:
oBaseDirPath = "D:\\ISS_project\dataset\\face_dataset_RGB\\"

def get_data(oDirList):
    oFinalImagePath = []
    oFinalGroundTruth = []
    for directory in oDirList:
        oDirPath = os.path.join(oBaseDirPath,directory)
        #read groundtruth
        oGrthFilePath = os.path.join(oDirPath,"angles.txt")
        oGetGroundtruth = pd.read_csv(oGrthFilePath, sep="\t", header=None)
        #print(oGetGroundtruth[3][3])
        #get list of all images
        oImagePathList = glob(oDirPath+"\\*.png")
        #save image path and corresponding groundtruth to final list
        for imgPath in oImagePathList:
            oImageName = imgPath.split("\\")[-1]
            oImageId = int(oImageName.split("_")[1])
            oGTAngles = [oGetGroundtruth[3][oImageId],-1 * oGetGroundtruth[1][oImageId],-1 * oGetGroundtruth[2][oImageId]]
            oFinalImagePath.append(imgPath)
            oFinalGroundTruth.append(oGTAngles)
    return oFinalImagePath,oFinalGroundTruth

# Testing Groundtruth by Displaying the vectors

In [3]:
# Calculates Rotation Matrix given euler angles.
def eulerAnglesToRotationMatrix(theta) :
    R_x = np.identity(3)
    R_y = np.identity(3)
    R_z = np.identity(3)
     
    R_x = np.array([[1,0,0],
                    [0,math.cos(theta[0]), -math.sin(theta[0]) ],
                    [0,math.sin(theta[0]), math.cos(theta[0])]
                    ])
         
         
                     
    R_y = np.array([[math.cos(theta[1]),0,math.sin(theta[1])],
                    [0,1,0],
                    [-math.sin(theta[1]),0,math.cos(theta[1])]
                    ])
                 
    R_z = np.array([[math.cos(theta[2]),-math.sin(theta[2]),0],
                    [math.sin(theta[2]),math.cos(theta[2]),0],
                    [0,0,1]
                    ])
                     
                     
    R = np.dot(R_z, np.dot( R_y, R_x ))
 
    return R

#display headpose
def displayHeadpose(oImgNp, oAnglesNp,oOffset = 0):
    oHeadposeRotationMat = eulerAnglesToRotationMatrix(oAnglesNp)
    oPointsNp = np.array([(50, 0 ,0),
                         (0, 50, 0),
                         (0, 0, 50)])
    oPointsNp = np.dot(oHeadposeRotationMat, oPointsNp)
    oOriginT = (int(oImgNp.shape[1]/2 + oOffset), int(oImgNp.shape[0]/2))
    oLineXT = (int(oOriginT[0] - oPointsNp[0,0]), int(oOriginT[1] - oPointsNp[1,0]))
    oLineYT = (int(oOriginT[0] - oPointsNp[0,1]), int(oOriginT[1] - oPointsNp[1,1]))
    oLineZT = (int(oOriginT[0] - oPointsNp[0,2]), int(oOriginT[1] - oPointsNp[1,2]))
    cv2.line(oImgNp, oOriginT, oLineXT, (0,0,255),3)
    cv2.line(oImgNp, oOriginT, oLineYT, (0,255,0),3)
    cv2.line(oImgNp, oOriginT, oLineZT, (255,0,0),3)
    return oImgNp


    

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (100,100))

for i in range(0,len(oFinalImagePath)):
    oImage = cv2.imread(oFinalImagePath[i])
    #oImage = cv2.cvtColor(oImage, cv2.COLOR_BGR2RGB)
    oGT = [oFinalGroundTruth[i][0] * np.pi/180,oFinalGroundTruth[i][1] * np.pi/180,oFinalGroundTruth[i][2] * np.pi/180]
    oDisplayImage = displayHeadpose(oImage,oGT)
    out.write(oDisplayImage)
    
    #plt.imshow(oDisplayImage)
    #plt.show()
out.release()

# Split Data into Train and Validation

In [5]:
# split into train test sets
#list the directory name
oDirList = os.listdir(oBaseDirPath)
#X_train, X_test = train_test_split(oDirList, test_size=0.2,random_state=42)
X_train = oDirList[0:80]
X_test = oDirList[80:100]
oTrainFinalImagePath,oTrainFinalGroundTruth= get_data(X_train)
oValFinalImagePath,oValFinalGroundTruth= get_data(X_test)
#print("Total data count :",len(oFinalImagePath))
print("No: of training data : ",X_train)
print("No: of Validation data : ",X_test)


No: of training data :  ['001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080']
No: of Validation data :  ['081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100']


# Copy data into train and Validation folders

In [6]:

#destination directories 
oDestTrainDir = "D:\\ISS_project\\dataset\\prepared_data\\train\\"
#copy training images and Ground truth
for i in range(0,len(oTrainFinalImagePath)):
    #file name
    oImageName = oTrainFinalImagePath[i].split("\\")[-1]
    oImageName = oImageName.split(".png")[0]
    ImageDirName = oTrainFinalImagePath[i].split("\\")[-2]
    oDestinationImageName = os.path.join(oDestTrainDir,ImageDirName + "_" + oImageName + ".png")
    #copy file
    shutil.copy(oTrainFinalImagePath[i], oDestinationImageName)
    #write groundtruth into textfile
    oGTpath = os.path.join(oDestTrainDir,ImageDirName + "_" + oImageName + "_angles.txt")
    with open(oGTpath, 'w') as f:
        f.write(json.dumps(oTrainFinalGroundTruth[i]))
        
    

In [7]:
#destination directories 
oDestValDir = "D:\\ISS_project\\dataset\\prepared_data\\val\\"
#copy validation images and Ground truth
for i in range(0,len(oValFinalImagePath)):
    #file name
    oImageName = oValFinalImagePath[i].split("\\")[-1]
    oImageName = oImageName.split(".png")[0]
    ImageDirName = oValFinalImagePath[i].split("\\")[-2]
    oDestinationImageName = os.path.join(oDestValDir,ImageDirName + "_" + oImageName + ".png")
    #copy file
    shutil.copy(oValFinalImagePath[i], oDestinationImageName)
    #write groundtruth into textfile
    oGTpath = os.path.join(oDestValDir,ImageDirName + "_" + oImageName +"_angles.txt")
    with open(oGTpath, 'w') as f:
        f.write(json.dumps(oValFinalGroundTruth[i]))

# Prepare Biwi Dataset for Testing

In [26]:
oBaseDirPath = "D:\\ISS_project\\dataset\\biwi_face_dataset_RGB\\face_dataset_RGB\\"
#list the directory name
oDirList = os.listdir(oBaseDirPath)
oFinalImagePath = []
oFinalGroundTruth = []
for directory in oDirList:
    oDirPath = os.path.join(oBaseDirPath,directory)
    #read groundtruth
    oGrthFilePath = os.path.join(oDirPath,"angles.txt")
    oGetGroundtruth = pd.read_csv(oGrthFilePath, sep="\t", header=None)
    #print(oGetGroundtruth[3][5])
    #get list of all images
    oImagePathList = glob(oDirPath+"\\*.png")
    #save image path and corresponding groundtruth to final list
    for imgPath in oImagePathList:
        oImageName = imgPath.split("\\")[-1]
        oImageId = int(oImageName.split("_")[1])        
        #get GT array index
        oIndx = oGetGroundtruth[oGetGroundtruth[0] == oImageId].index.tolist()
        oGTAngles = [oGetGroundtruth[2][oIndx[0]],oGetGroundtruth[3][oIndx[0]],oGetGroundtruth[1][oIndx[0]]]
        oFinalImagePath.append(imgPath)
        oFinalGroundTruth.append(oGTAngles)

# Display the groundtruth

In [27]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_test.avi', fourcc, 20.0, (100,100))

for i in range(0,len(oFinalImagePath)):
    oImage = cv2.imread(oFinalImagePath[i])
    #oImage = cv2.cvtColor(oImage, cv2.COLOR_BGR2RGB)
    oGT = [oFinalGroundTruth[i][0] * np.pi/180,oFinalGroundTruth[i][1] * np.pi/180,oFinalGroundTruth[i][2] * np.pi/180]
    oDisplayImage = displayHeadpose(oImage,oGT)
    out.write(oDisplayImage)
    
    #plt.imshow(oDisplayImage)
    #plt.show()
out.release()

In [28]:
#destination directories 
oDestValDir = "D:\\ISS_project\\dataset\\prepared_data\\test\\"
#copy validation images and Ground truth
for i in range(0,len(oFinalImagePath)):
    #file name
    oImageName = oFinalImagePath[i].split("\\")[-1]
    oImageName = oImageName.split(".png")[0]
    ImageDirName = oFinalImagePath[i].split("\\")[-2]
    oDestinationImageName = os.path.join(oDestValDir,ImageDirName + "_" + oImageName + ".png")
    #copy file
    shutil.copy(oFinalImagePath[i], oDestinationImageName)
    #write groundtruth into textfile
    oGTpath = os.path.join(oDestValDir,ImageDirName + "_" + oImageName +"_angles.txt")
    with open(oGTpath, 'w') as f:
        f.write(json.dumps(oFinalGroundTruth[i]))